# MLS Player Categorization
The main objective of your project is to predict and categorize MLS players based on their performance into different tiers, such as elite, all-star, starter, rotation, and out of the league.

## Setup and Data    

In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [74]:
# Import CSV files into python
goalies_data = pd.read_csv("all_goalkeepers.csv")
player_data = pd.read_csv("all_players.csv")
table_data = pd.read_csv("all_tables.csv")
event_data = pd.read_csv("events.csv", low_memory = False)
match_data = pd.read_csv("matches.csv", low_memory = False)
awards_data = pd.read_csv("awards.csv")

## Exploratory Data Analysis & Cleaning
[FILL IN]

### Goalkeeper Data Transformation

In [75]:
# Goalkeeper Data
goalies_data.head()

,Player,Club,POS,GP,GS,MINS,SHTS,SV,GA,GAA,PKG/A,W,L,T,ShO,W%,Sv%,Year,Season
0,Mark Dodd,DAL,GK,31,31,2776,235,160,45,1.45,5/5,17,14,0,6,54.8,68.1,1996,reg
1,Tony Meola,MET,GK,29,29,2610,188,142,38,1.31,3/4,14,15,0,9,48.3,75.5,1996,reg
2,Mark Dougherty,TB,GK,28,28,2520,186,123,47,1.68,9/10,17,11,0,3,60.7,66.1,1996,reg
3,Jorge Campos,LA,GK,24,24,2160,133,100,27,1.13,0/1,13,8,0,4,54.2,75.2,1996,reg
4,Garth Lagerwey,KC,GK,23,22,1959,133,83,38,1.73,2/2,12,10,0,3,52.2,62.4,1996,reg


In [76]:
SELECT "PKG/A", COUNT(*) AS "CT" FROM goalies_data GROUP BY "PKG/A" 

,PKG/A,CT
0,0/1,62
1,4/5,26
2,4/7,6
3,7/7,3
4,5/8,2
5,8/9,1
6,6/9,1
7,5/7,7
8,7/9,2
9,7/10,2


In [77]:
# Split the "PKG/A" column into two separate columns using the '/' delimiter
goalies_data[['PK Made', 'PK Attempted']] = goalies_data['PKG/A'].str.split('/', expand=True)

# Convert the 'PK Made' and 'PK Attempted' columns to integers
goalies_data['PK Made'] = goalies_data['PK Made'].astype(int)
goalies_data['PK Attempted'] = goalies_data['PK Attempted'].astype(int)

# Calculate the PK Save Ratio as an integer and replace the "PKG/A" column
goalies_data['PKG/A'] = np.where(goalies_data['PK Attempted'] != 0, (goalies_data['PK Made'] / goalies_data['PK Attempted']), 0)

In [78]:
print(goalies_data.dtypes)

Player           object
Club             object
POS              object
GP                int64
GS                int64
MINS              int64
SHTS              int64
SV                int64
GA                int64
GAA             float64
PKG/A           float64
W                 int64
L                 int64
T                 int64
ShO               int64
W%              float64
Sv%             float64
Year              int64
Season           object
PK Made           int64
PK Attempted      int64
dtype: object


In [79]:
# Missing Data: Determine what columns are NaN or null in each dataset 
missing_goalies_data = goalies_data.columns[goalies_data.isna().any()].tolist()

print('Missing Goalie Column Data: ', missing_goalies_data)

Missing Goalie Column Data:  ['Club']


### Player Data Transformation

In [80]:
# Player Data
player_data.head()

,Player,Club,POS,GP,GS,MINS,G,A,SHTS,SOG,GWG,PKG/A,HmG,RdG,G/90min,SC%,GWA,HmA,RdA,A/90min,FC,FS,OFF,YC,RC,SOG%,Year,Season
0,Roy Lassiter,TB,F,30,30,2580,27,4,76,49,4,3/3,19,8,0.94,35.5,2,2,2,0.14,20,39,70,2,0,64.47,1996,reg
1,Raul Diaz Arce,DC,F,28,28,2351,23,2,100,49,4,4/4,15,8,0.88,23.0,0,0,2,0.08,32,26,35,6,1,49.00,1996,reg
2,Eduardo Hurtado,LA,F,26,26,2323,21,7,87,56,6,2/3,13,8,0.81,24.1,0,4,3,0.27,48,26,25,5,0,64.37,1996,reg
3,Preki,KC,M,32,32,2880,18,13,140,61,3,7/8,14,4,0.56,12.9,2,9,4,0.41,26,44,7,3,0,43.57,1996,reg
4,Brian McBride,CLB,F,28,28,2307,17,3,79,44,3,3/4,12,5,0.66,21.5,0,2,1,0.12,21,46,10,0,0,55.70,1996,reg


In [81]:
# Missing Data: Determine what columns are NaN or null in each dataset 
missing_player_data = player_data.columns[player_data.isna().any()].tolist()

print('Missing Player Column Data: ', missing_player_data)

Missing Player Column Data:  ['Club', 'SOG%']


In [82]:
SELECT * FROM player_data WHERE Club IS NULL AND Season != 'post'

,Player,Club,POS,GP,GS,MINS,G,A,SHTS,SOG,GWG,PKG/A,HmG,RdG,G/90min,SC%,GWA,HmA,RdA,A/90min,FC,FS,OFF,YC,RC,SOG%,Year,Season


In [83]:
SELECT * FROM player_data WHERE "SOG%" IS NULL

,Player,Club,POS,GP,GS,MINS,G,A,SHTS,SOG,GWG,PKG/A,HmG,RdG,G/90min,SC%,GWA,HmA,RdA,A/90min,FC,FS,OFF,YC,RC,SOG%,Year,Season
0,Petter Villegas,MET,M,3,2,165,0,1,0,0,0,0/0,0,0,0.0,0.0,1,0,1,0.55,2,5,1,0,0,NaN,1996,reg
1,Obi Monome,CLB,M,1,0,26,0,1,0,0,0,0/0,0,0,0.0,0.0,0,0,1,3.46,0,0,0,0,0,NaN,1996,reg
2,Andrew Shue,LA,F,5,0,96,0,1,0,0,0,0/0,0,0,0.0,0.0,0,0,1,0.94,1,1,0,0,0,NaN,1996,reg
3,George Gelnovatch,DC,M,2,2,135,0,0,0,0,0,0/0,0,0,0.0,0.0,0,0,0,0.00,1,4,0,0,0,NaN,1996,reg
4,Said Fazlagic,DC,D,2,1,136,0,0,0,0,0,0/0,0,0,0.0,0.0,0,0,0,0.00,3,1,0,1,0,NaN,1996,reg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5065,Michael Halliday,None,D,0,0,0,0,0,0,0,0,0/0,0,0,0.0,0.0,0,0,0,0.00,0,0,0,0,0,NaN,2020,post
5066,Callum Montgomery,None,D,0,0,0,0,0,0,0,0,0/0,0,0,0.0,0.0,0,0,0,0.00,0,0,0,0,0,NaN,2020,post
5067,Pablo Bonilla,None,D,1,1,94,0,0,0,0,0,0/0,0,0,0.0,0.0,0,0,0,0.00,1,2,0,0,0,NaN,2020,post
5068,Justin Che,None,D,0,0,0,0,0,0,0,0,0/0,0,0,0.0,0.0,0,0,0,0.00,0,0,0,0,0,NaN,2020,post


### Table Data Transformation

In [84]:
# Table Data
table_data.head()

,Pos,Team,GP,W,L,SW,GF,GA,GD,Pts,Qualification,Conference,Year,SL,D,Head-to-head,PPG
0,1.0,Tampa Bay Mutiny,32,19,12,1.0,66,51,+15,58,Playoffs,Eastern Conference,1996,NaN,NaN,NaN,NaN
1,2.0,D.C. United,32,15,16,1.0,62,56,+6,46,Playoffs,Eastern Conference,1996,NaN,NaN,NaN,NaN
2,3.0,NY/NJ MetroStars,32,12,17,3.0,45,47,−2,39,Playoffs,Eastern Conference,1996,NaN,NaN,NaN,NaN
3,4.0,Columbus Crew,32,11,17,4.0,59,60,−1,37,Playoffs,Eastern Conference,1996,NaN,NaN,NaN,NaN
4,5.0,New England Revolution,32,9,17,6.0,43,56,−13,33,NaN,Eastern Conference,1996,NaN,NaN,NaN,NaN


In [85]:
# Missing Data: Determine what columns are NaN or null in each dataset 
missing_table_data = table_data.columns[table_data.isna().any()].tolist()

print('Missing Table Column Data: ', missing_table_data)

Missing Table Column Data:  ['Pos', 'SW', 'Qualification', 'SL', 'D', 'Head-to-head', 'PPG']


### Event Data Transformation

In [86]:
# Event Data
event_data.head()

,id,Time,Event
0,14876,-,no commentary
1,14877,-,no commentary
2,14879,-,no commentary
3,14878,-,no commentary
4,14880,-,no commentary


### Match Data Transformation

In [87]:
# Match Data
match_data = match_data[['id', 'home', 'away', 'date', 'year', 'part_of_competition', 'game_status', 'home_score', 'away_score']]

match_data.head(5)

,id,home,away,date,year,part_of_competition,game_status,home_score,away_score
0,NaN,New England,San Jose,7/31/1996,1996,Regular Season,FT,2,0
1,NaN,Dallas,Colorado,6/15/1996,1996,Regular Season,FT,1,1
2,NaN,Colorado,D.C. United,8/29/1996,1996,Regular Season,FT,1,2
3,NaN,LA Galaxy,New England,8/8/1996,1996,Regular Season,FT,1,0
4,NaN,New England,D.C. United,7/20/1996,1996,Regular Season,FT,2,0


In [88]:
SELECT year, COUNT(*) FROM match_data WHERE id IS NULL GROUP BY "year"

,year,count_star()
0,1997,173
1,1999,208
2,1998,206
3,2000,209
4,1996,177


In [89]:
# Missing Data: Determine what columns are NaN or null in each dataset 
missing_match_data = match_data.columns[match_data.isna().any()].tolist()

print('Missing Match Column Data: ', missing_match_data)

Missing Match Column Data:  ['id']


## MLS Club / Team Dictionary

In [90]:
# Generate team name to 3-4 letter abbreviation mapping
# Reference: ChatGPT (also: https://www.shrpsports.com/mls/explain.htm)
# Update list: ShrpSports (https://www.shrpsports.com/mls/explain.htm)
mls_teams = {
    "ATL": "Atlanta United FC",
    "AUS": "Austin FC",
    "CHI": "Chicago Fire FC",
    "CHV": "Chivas USA",
    "CIN": "FC Cincinnati",
    "COL": "Colorado Rapids",
    "CLB": "Columbus Crew",
    "CLT": "Charlotte FC",
    "DBU": "Dallas Burn",
    "DC": "D.C. United",
    "DAL": "FC Dallas",
    "HOU": "Houston Dynamo FC",
    "MIA": "Inter Miami CF",
    "MIF": "Miami Fusion",
    "KWZ": "Kansas City Wiz",
    "KCW": "Kansas City Wizards",
    "LAG": "LA Galaxy",
    "LAFC": "Los Angeles FC",
    "MET": "NY/NJ MetroStars",
    "MIN": "Minnesota United FC",
    "MTI": "Montreal Impact",
    "MTL": "CF Montréal",
    "NSH": "Nashville SC",
    "NE": "New England Revolution",
    "NYCFC": "New York City FC",
    "NYRB": "New York Red Bulls",
    "ORL": "Orlando City SC",
    "PHI": "Philadelphia Union",
    "POR": "Portland Timbers",
    "RSL": "Real Salt Lake",
    "SJ": "San Jose Earthquakes",
    "SEA": "Seattle Sounders FC",
    "SKC": "Sporting Kansas City",
    "STL": "St. Louis City SC",
    "TOR": "Toronto FC",
    "TB": "Tampa Bay Mutiny",
    "VAN": "Vancouver Whitecaps FC"
}

# Accessing the data dictionary
team_abbreviation = "SEA"
team_name = mls_teams.get(team_abbreviation, "Team not found")

# Print the team name for the given abbreviation
print(f"The team abbreviation '{team_abbreviation}' corresponds to the team '{team_name}'")

The team abbreviation 'SEA' corresponds to the team 'Seattle Sounders FC'


## PART 1: Awards 

Top Awards based on positions include: 
- Goalkeeper: GK of the Year Award
- Midfielder: 
- Defender: Defender of the Year Award
- Forward: 

In [91]:
# Review awards data
awards_data.head()

,Year,Player,Nation,Team,ESPY Winner,ESPY Finalist,Best XI,Comeback Player of the Year Winner,Defender Award Other Finalists,Defender of the Year Winner,GK of the Year Rank,GK of the Year Finalist,Goal of the Year,Golden Boot,No of Months,Player of the Month,Landon Donovan MVP Winner,Landon Donovan MVP Finalist,All-Star Game MVP,Newcomer of the Year,Position,Draft Pick,Draft Class,College,Age,Young Player of the Year
0,2006,Landon Donovan,USA,LA Galaxy,1.0,1.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
1,2007,Landon Donovan,USA,LA Galaxy,1.0,1.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
2,2008,David Beckham,ENG,LA Galaxy,1.0,1.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
3,2009,Landon Donovan,USA,LA Galaxy,1.0,1.0,0.0,0.0,NaN,0.0,NaN,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
4,2010,Landon Donovan,USA,LA Galaxy,1.0,1.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0


In [92]:
# CHECK: Merge player_data w/ awards_data based on year (see if teams do not match)
